In [29]:
#first I will import Libraries
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd
! pip install shapely
from shapely.geometry import Point
import matplotlib.pyplot as plt
! pip install geopandas
import geopandas as gpd
import seaborn as sns

print('Libraries imported.')

     |████████████████████████████████| 972kB 7.0MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 35.3MB/s eta 0:00:01
Libraries imported.


In [2]:
#Getting the data from url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

In [3]:
#First 5
data.head()

,Borough,Neighbourhood,Postal Code
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A


In [5]:
#Choosing only data where field Borough doesn't have not assigned value
clean_data = data[data['Borough'] != 'Not assigned']

In [6]:
#Grouping Data
clean_data = clean_data.groupby(['Postal Code','Borough'], as_index=False).agg(','.join)

In [7]:

clean_data.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
#Replacing values in Neighbourhood field with Borough where Neighbourhood is not assigned
clean_data['Neighbourhood'] = np.where(clean_data['Neighbourhood'] == 'Not assigned', clean_data['Borough'], clean_data['Neighbourhood'])

In [9]:
#Shape of Data
raw_data_selected.shape

(103, 3)

Question 2: Geocoder package/csv file used to creat dataframe

In [10]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)

In [11]:
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# Renaming the columns
geospatial_data.columns = ['Postal Code', 'Latitude', 'Longitude']

In [19]:
geospatial_data.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [20]:
#Merging dataframes
merged_data = pd.merge(clean_data, geospatial_data, on='Postal Code')

In [21]:
merged_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Q3

In [22]:
merged_data['Coordinates'] = list(zip(merged_data['Latitude'], merged_data['Longitude']))

In [23]:
merged_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Coordinates
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,"(43.806686299999996, -79.19435340000001)"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,"(43.7845351, -79.16049709999999)"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,"(43.7635726, -79.1887115)"
3,M1G,Scarborough,Woburn,43.770992,-79.216917,"(43.7709921, -79.21691740000001)"
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,"(43.773136, -79.23947609999999)"


In [30]:
merged_data['Coordinates'] = merged_data['Coordinates'].apply(Point)

In [31]:
gdf = gpd.GeoDataFrame(merged_data, geometry='Coordinates')

In [32]:
gdf.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Coordinates
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,POINT (43.80669 -79.19435)
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,POINT (43.78454 -79.16050)
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,POINT (43.76357 -79.18871)
3,M1G,Scarborough,Woburn,43.770992,-79.216917,POINT (43.77099 -79.21692)
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,POINT (43.77314 -79.23948)
